In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [3]:
#load the MNIST data set
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
np.isnan(x_train).any()


np.False_

In [5]:
np.isnan(x_test).any()


np.False_

In [6]:
# Normalization and Reshaping
input_shape = (28, 28, 1)

x_train=x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train=x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_test=x_test/255.0

In [7]:
# Label Encoding
y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

In [10]:
#  Define the Model
batch_size = 64
num_classes = 10
epochs = 2

In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=input_shape),  # Use Input layer here
    tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(strides=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(epsilon=1e-08), loss='categorical_crossentropy', metrics=['acc'])


In [12]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.995):
      print("\nReached 99.5% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [14]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=1,
                    validation_split=0.1,
                    callbacks=[callbacks])

844/844 ━━━━━━━━━━━━━━━━━━━━ 72s 85ms/step - acc: 0.9507 - loss: 0.1654 - val_acc: 0.9878 - val_loss: 0.0443


In [15]:
test_loss, test_acc = model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - acc: 0.9809 - loss: 0.0626


In [16]:
print(test_acc)
print(test_loss)

0.9846000075340271
0.05006321147084236


In [19]:
# Save the model in the new native Keras format
model.save('model/mnist.keras')  # Recommended format



In [20]:
from tensorflow.keras.preprocessing import image

In [27]:
# Load your input image and preprocess it
img_path = 'seven.png'  # Replace with the path to your image
img = image.load_img(img_path, target_size=(28, 28), color_mode='grayscale')  # Adjust target_size based on your input_shape
img_array = image.img_to_array(img)

# Normalize the image to match the scale your model was trained on
img_array = img_array / 255.0  # Assuming your model was trained on normalized data

# Reshape to match the input shape (batch_size, height, width, channels)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

In [28]:
loaded_model = tf.keras.models.load_model('model/mnist.keras')

In [29]:
predictions = model.predict(img_array)

# The result is an array of probabilities for each class
predicted_class = np.argmax(predictions, axis=-1)

print(f'Predicted class: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Predicted class: [7]
